In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/VOC/VOCdevkit/VOC2007

/content/drive/MyDrive/VOC/VOCdevkit/VOC2007


In [ ]:
!pip install labelme==3.16.7

     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 8.3 MB 15.0 MB/s 
     |████████████████████████████████| 317 kB 56.6 MB/s 
     |████████████████████████████████| 59.9 MB 35 kB/s 
  Created wheel for labelme: filename=labelme-3.16.7-py3-none-any.whl size=1462722 sha256=787efb53a3271f475904f458a9fb73e036bcf180b78c0c9328fd60858bca574a
  Stored in directory: /root/.cache/pip/wheels/93/da/63/a9473a13842477f304ade974dadb454f66e4886b576e2cb254
Successfully built labelme


In [ ]:
#this part is transfer the json to jpg. json file is produced using "labelme". This part just need to run once. 
#After running, SegmentationClass folder will include label image.
'''
import base64
import json
import os
import os.path as osp

import numpy as np
import PIL.Image
from labelme import utils

'''
'''
if __name__ == '__main__':
    jpgs_path   = "/JPEGImages"
    pngs_path   = "/SegmentationClass"
    #classes     = ["_background_","aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
    classes     = ["_background_","iris"]
    
    count = os.listdir("./before/") 
    for i in range(0, len(count)):
        path = os.path.join("./before", count[i])

        if os.path.isfile(path) and path.endswith('json'):
            data = json.load(open(path))
            
            if data['imageData']:
                imageData = data['imageData']
            else:
                imagePath = os.path.join(os.path.dirname(path), data['imagePath'])
                with open(imagePath, 'rb') as f:
                    imageData = f.read()
                    imageData = base64.b64encode(imageData).decode('utf-8')

            img = utils.img_b64_to_arr(imageData)
            label_name_to_value = {'_background_': 0}
            for shape in data['shapes']:
                label_name = shape['label']
                if label_name in label_name_to_value:
                    label_value = label_name_to_value[label_name]
                else:
                    label_value = len(label_name_to_value)
                    label_name_to_value[label_name] = label_value
            
            # label_values must be dense
            label_values, label_names = [], []
            for ln, lv in sorted(label_name_to_value.items(), key=lambda x: x[1]):
                label_values.append(lv)
                label_names.append(ln)
            assert label_values == list(range(len(label_values)))
            
            lbl = utils.shapes_to_label(img.shape, data['shapes'], label_name_to_value)
            
                
            PIL.Image.fromarray(img).save(osp.join('/content/drive/MyDrive/VOC/VOCdevkit/VOC2007/test/JPEGImages', count[i].split(".")[0]+'.jpg'))

            new = np.zeros([np.shape(img)[0],np.shape(img)[1]])
            for name in label_names:
                index_json = label_names.index(name)
                index_all = classes.index(name)
                new = new + index_all*(np.array(lbl) == index_json)

            utils.lblsave(osp.join('/content/drive/MyDrive/VOC/VOCdevkit/VOC2007/test/SegmentationClass', count[i].split(".")[0]+'.png'), new)
            print('Saved ' + count[i].split(".")[0] + '.jpg and ' + count[i].split(".")[0] + '.png')
            
'''

Saved S1060R01.jpg and S1060R01.png
Saved S1060L10.jpg and S1060L10.png
Saved S1060R03.jpg and S1060R03.png
Saved S1060L09.jpg and S1060L09.png
Saved S1060R04.jpg and S1060R04.png
Saved S1060L07.jpg and S1060L07.png
Saved S1060R02.jpg and S1060R02.png
Saved S1060L08.jpg and S1060L08.png
Saved S1056L03.jpg and S1056L03.png
Saved S1056L02.jpg and S1056L02.png
Saved S1056L01.jpg and S1056L01.png
Saved S1060R10.jpg and S1060R10.png
Saved S1059L02.jpg and S1059L02.png
Saved S1059L01.jpg and S1059L01.png
Saved S1059L03.jpg and S1059L03.png
Saved S1059L04.jpg and S1059L04.png
Saved S1058R05.jpg and S1058R05.png
Saved S1058R04.jpg and S1058R04.png
Saved S1058R03.jpg and S1058R03.png
Saved S1058R02.jpg and S1058R02.png
Saved S1060R09.jpg and S1060R09.png
Saved S1060R08.jpg and S1060R08.png
Saved S1060R06.jpg and S1060R06.png
Saved S1060R07.jpg and S1060R07.png
Saved S1060L06.jpg and S1060L06.png
Saved S1060L05.jpg and S1060L05.png
Saved S1060L04.jpg and S1060L04.png
Saved S1060L03.jpg and S1060

In [ ]:
from PIL import Image

# this part is used to resize image
def keep_image_size_open(path, size=(256, 256)):
    img = Image.open(path)
    temp = max(img.size)
    mask = Image.new('RGB', (temp, temp), (0, 0, 0))
    mask.paste(img, (0, 0))
    mask = mask.resize(size)
    return mask


In [ ]:
import os

from torch.utils.data import Dataset

from torchvision import transforms
transform=transforms.Compose([
    transforms.ToTensor()
])
# dataset setting. dataset will find the file in ./SegmentationClass/ and ./JPEGImages/
class MyDataset(Dataset):
    def __init__(self,path):
        self.path=path
        self.name=os.listdir(os.path.join(path,'SegmentationClass'))

    def __len__(self):
        return len(self.name)

    def __getitem__(self, index):
        segment_name=self.name[index]  #xx.png
        segment_path=os.path.join(self.path,'SegmentationClass',segment_name)
        image_path=os.path.join(self.path,'JPEGImages',segment_name.replace('png','jpg'))
        segment_image=keep_image_size_open(segment_path)
        image=keep_image_size_open(image_path)
        return transform(image),transform(segment_image),segment_name

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
# U-net architecture
class Conv_Block(nn.Module):
    def __init__(self,in_channel,out_channel):
        super(Conv_Block, self).__init__()
        self.layer=nn.Sequential(
            nn.Conv2d(in_channel,out_channel,3,1,1,padding_mode='reflect',bias=False),
            nn.BatchNorm2d(out_channel),
            nn.Dropout2d(0.3),
            nn.LeakyReLU(),
            nn.Conv2d(out_channel, out_channel, 3, 1, 1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(out_channel),
            nn.Dropout2d(0.3),
            nn.LeakyReLU()
        )
    def forward(self,x):
        return self.layer(x)


class DownSample(nn.Module):
    def __init__(self,channel):
        super(DownSample, self).__init__()
        self.layer=nn.Sequential(
            nn.Conv2d(channel,channel,3,2,1,padding_mode='reflect',bias=False),
            nn.BatchNorm2d(channel),
            nn.LeakyReLU()
        )
    def forward(self,x):
        return self.layer(x)


class UpSample(nn.Module):
    def __init__(self,channel):
        super(UpSample, self).__init__()
        self.layer=nn.Conv2d(channel,channel//2,1,1)
    def forward(self,x,feature_map):
        up=F.interpolate(x,scale_factor=2,mode='nearest')
        out=self.layer(up)
        return torch.cat((out,feature_map),dim=1)


class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.c1=Conv_Block(3,64)
        self.d1=DownSample(64)
        self.c2=Conv_Block(64,128)
        self.d2=DownSample(128)
        self.c3=Conv_Block(128,256)
        self.d3=DownSample(256)
        self.c4=Conv_Block(256,512)
        self.d4=DownSample(512)
        self.c5=Conv_Block(512,1024)
        self.u1=UpSample(1024)
        self.c6=Conv_Block(1024,512)
        self.u2 = UpSample(512)
        self.c7 = Conv_Block(512, 256)
        self.u3 = UpSample(256)
        self.c8 = Conv_Block(256, 128)
        self.u4 = UpSample(128)
        self.c9 = Conv_Block(128, 64)
        
        self.out=nn.Conv2d(64,3,3,1,1)
        self.Th=nn.Sigmoid()

    def forward(self,x):
        R1=self.c1(x)
        R2=self.c2(self.d1(R1))
        R3 = self.c3(self.d2(R2))
        R4 = self.c4(self.d3(R3))
        R5 = self.c5(self.d4(R4))
        O1=self.c6(self.u1(R5,R4))
        O2 = self.c7(self.u2(O1, R3))
        O3 = self.c8(self.u3(O2, R2))
        O4 = self.c9(self.u4(O3, R1))

        return self.Th(self.out(O4))



In [ ]:
import os

from torch import nn,optim
import torch
from torch.utils.data import DataLoader

from torchvision.utils import save_image
# training
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weight_path='params/unet.pth'# used to save parameters
data_path=r''# load input image
save_path='train_image'# save forward pass result
if __name__ == '__main__':
    data_loader=DataLoader(MyDataset(data_path),batch_size=2,shuffle=True)
    net=UNet().to(device)
    if os.path.exists(weight_path):
        net.load_state_dict(torch.load(weight_path))
        print('successful load weight！')
    else:
        print('not successful load weight')

    opt=optim.Adam(net.parameters())
    loss_fun=nn.BCELoss()

    epoch=1
    while True:
        for i,(image,segment_image,name) in enumerate(data_loader):
            image, segment_image=image.to(device),segment_image.to(device)

            out_image=net(image)
            train_loss=loss_fun(out_image,segment_image)

            opt.zero_grad()
            train_loss.backward()
            opt.step()

            if i%5==0:
                print(f'{epoch}-{i}-train_loss===>>{train_loss.item()}')

            if i%50==0:
                torch.save(net.state_dict(),weight_path)

            _image=image[0]
            _segment_image=segment_image[0]
            _out_image=out_image[0]
            
            img=torch.stack([_image,_segment_image,_out_image],dim=0)
            save_image(img,f'{save_path}/{i}.png')

        epoch+=1



successful load weight！
1-0-train_loss===>>0.06313926726579666
1-5-train_loss===>>0.06090247631072998
1-10-train_loss===>>0.05681052803993225
1-15-train_loss===>>0.06394992023706436
1-20-train_loss===>>0.06036822497844696
1-25-train_loss===>>0.06287642568349838
1-30-train_loss===>>0.06424080580472946
1-35-train_loss===>>0.05407027527689934
1-40-train_loss===>>0.05622170493006706
1-45-train_loss===>>0.06048991158604622
1-50-train_loss===>>0.07242171466350555
1-55-train_loss===>>0.08518513292074203
1-60-train_loss===>>0.06244334578514099
1-65-train_loss===>>0.06358705461025238
1-70-train_loss===>>0.08223915845155716
1-75-train_loss===>>0.0714908167719841
1-80-train_loss===>>0.058198779821395874
1-85-train_loss===>>0.06245889887213707
1-90-train_loss===>>0.0625138059258461
1-95-train_loss===>>0.0562090240418911
1-100-train_loss===>>0.054260507225990295
1-105-train_loss===>>0.07406994700431824
1-110-train_loss===>>0.06325371563434601
1-115-train_loss===>>0.05830897390842438
1-120-train_los

In [ ]:
weight_path='params/unet.pth'# load parameter
data_path=r'test'# find input image in under this folder
save_path='train_image'# save predict(stack 3 images(original, label, predict))
save_output_path='show'#save predicted image only
if __name__ == '__main__':
    testdata_loader=DataLoader(MyDataset(data_path),batch_size=1,shuffle=False)
    

   
    for i,(image,segment_image,name) in enumerate(testdata_loader):
      image, segment_image=image.to(device),segment_image.to(device)
      name1 = ''.join(name)
      name1.replace("(","").replace(")","").replace("'","").replace(",","")
      print(name1)
      out_image=net(image)
      _image=image[0]
      _segment_image=segment_image[0]
      _out_image=out_image[0]
      save_image(_out_image,f'{save_output_path}/{name1}')#added
      img=torch.stack([_image,_segment_image,_out_image],dim=0)
      save_image(img,f'{save_path}/{i}.png')


S1060R01.png
S1060L10.png
S1060R03.png
S1060L09.png
S1060R04.png
S1060L07.png
S1060R02.png
S1060L08.png
S1056L03.png
S1056L02.png
S1056L01.png
S1060R10.png
S1059L02.png
S1059L01.png
S1059L03.png
S1059L04.png
S1058R05.png
S1058R04.png
S1058R03.png
S1058R02.png
S1060R09.png
S1060R08.png
S1060R06.png
S1060R07.png
S1060L06.png
S1060L05.png
S1060L04.png
S1060L03.png
S1060L01.png
S1060L02.png
S1057R02.png
S1057R01.png
S1057R05.png
S1057R04.png
S1058R01.png
S1057R03.png
S1059L05.png
S1060R05.png
S1056L05.png
S1056L04.png


In [ ]:
#compute IoU
import numpy as np
pre_path='show/'
label_path=''
count = os.listdir(pre_path)
TP = 0
FP = 0
TN = 0
FN = 0

for name in count[0:1]:
  label_image = os.path.join('test/SegmentationClass',name)
  pre_image = os.path.join('show',name)

  img1 = np.array(keep_image_size_open(label_image))
  img1_gray = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
  ret1,img1_binary=cv.threshold(img1_gray,10,255,cv.THRESH_BINARY)

  img2 = cv.imread(pre_image)
  img2_gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
  ret2,img2_binary=cv.threshold(img2_gray,10,255,cv.THRESH_BINARY) 
  
  height, width = img1_binary.shape
  print(label_image)
  print(pre_image)
  for i in range(height-32):
    for j in range(width):
      if img1_binary[i][j] == 255 and img2_binary[i][j] == 255:
        TP = TP+1
      elif img1_binary[i][j] == 0 and img2_binary[i][j] == 255:
        FP = FP+1
      elif img1_binary[i][j] == 255 and img2_binary[i][j] == 0:
        TN = TN+1
      elif img1_binary[i][j] == 0 and img2_binary[i][j] == 0:
        FN = FN+1 
  print(TP,FP,TN,FN)  
IOU = TP/(TP+FP+TN)
print(IOU)   


test/SegmentationClass/S1056L02.png
show/S1056L02.png
18981 2386 940 35037
0.850898820997893
